In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..', '..', '..'))

if module_path not in sys.path:
    sys.path.append(os.path.join(module_path))

from functions.augmentation import get_augmentation_layers
from functions.loading_data import load_dataset, prepare_train_dataset, prepare_valid_dataset
from functions.model_initialization import calculate_initial_biases
from functions.model_running import \
    run_model, \
    get_run_number, \
    increment_run_number,\
    preserve_best_runs
from functions.plotting import plot_single_output_history
from models.inception_resnet_v2_models import get_model_partly_frozen

WIDTH = 150
HEIGHT = 150

In [ ]:
data_dir = os.path.join(
    '..',
    '..',
    '..',
    '..',
    '..',
    'data2',
    'images_original_inception_resnet_v2_150x150_categorized_no_test')
train_dataset = load_dataset(HEIGHT, WIDTH, data_dir, 'training')
valid_dataset = load_dataset(HEIGHT, WIDTH, data_dir, 'validation')

In [ ]:
MODEL_NAME_BASE = 'inception_dealing_with_imbalance_bias_only_2_only_bottom_half_layers_trainable'
num_classes = len(train_dataset.class_names)
run_number = get_run_number(MODEL_NAME_BASE)

In [ ]:
data_augmentation = get_augmentation_layers()
train_dataset = prepare_train_dataset(train_dataset, data_augmentation)
valid_dataset = prepare_valid_dataset(valid_dataset)

In [ ]:
model_name = f'{MODEL_NAME_BASE}_{run_number}'
history = run_model(
    train_dataset,
    valid_dataset,
    lambda: get_model_partly_frozen(HEIGHT, WIDTH, num_classes),
    os.path.join('..', '..', '..', '..', 'tmp_models', model_name + '_{epoch}'),
    os.path.join('..', '..', '..', '..', 'tensor_logs', model_name),
    reduction_patience=10,
    stopping_patience=20)

In [ ]:
increment_run_number(MODEL_NAME_BASE)
preserve_best_runs(
    os.path.join('..', '..', '..', '..', 'tmp_models'),
    os.path.join('..', '..', '..', '..', 'models'))
plot_single_output_history(history.history)